In [1]:
import os
import sys
import urllib.request
import json
import pandas as pd
import re
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup
import requests

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 키워드 입력
query = input("검색어 : ")


# Chrome 실행
driver = webdriver.Chrome()
# url = "https://section.blog.naver.com/" 
naver_url = f'https://search.naver.com/search.naver?ssc=tab.blog.all&query={query}&sm=tab_opt&nso=so%3Ar%2Cp%3A1w'
driver.get(naver_url)
time.sleep(2)


# Pagedown
n = 0
while n < 200: 
    try: 
        element = driver.find_element(By.TAG_NAME,'body')
        element.send_keys(Keys.END)
        n = n+1
        time.sleep(0.05)
    except : 
        print('finish')
        break 
    
    
# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')


# 블로그 제목, url 가져오기
articles = ".title_link"
articles_data = driver.find_elements(By.CSS_SELECTOR, articles)
titles = []
urls = []

for article in articles_data :
    title = article.text
    url = article.get_attribute('href')
    titles.append(title)
    urls.append(url)


# 연관검색어 리스트에 담기
sub_keywords_data = soup.select('div.related_srch > ul > li')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print(*sub_keywords)


# driver.close()
        

In [ ]:
df = pd.DataFrame({"title" : titles, "url" : urls})
df

In [ ]:
def temp(driver):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')

    # 연관검색어 리스트에 담기
    return soup.select('div > div.detail_box > div.title_area')

# NAVER BLOG 
- title/url 

In [7]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(2)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
end_page
contents_num = 7

blog_titles = []
blog_urls = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
    
    # 블로그 제목, url 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_titles.append(title)
        blog_urls.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_titles)
print(blog_urls)
    
# for page_num in range(1, end_page) : 
#     search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
#     driver.get(search_url)
#     time.sleep(0.5)
    
#     # for i in range(5):
#     #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     #     time.sleep(0.05)
    
#     html = driver.page_source
#     soup = BeautifulSoup(html, "html.parser")
    
    
#     # 블로그 제목, url 가져오기
#     posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
#     blog_titles = []
#     blog_urls = []

#     for post in posts_data :
#         title=post.get_text().replace('\n','').strip()
#         href=post.attrs['href']
#         titles.append(title)
#         urls.append(href)



2024-04-26 2024-05-03
총 블로그글 개수 :  12,046건
연관검색어: 
강남역 피부과강남역 카페강남역 놀거리강남역 미용실강남역 혼밥강남역 나세르강남역 브런치강남역 파스타강남역산부인과강남역 고기집강남역 삼겹살강남역 이자카야강남역 라인의원강남역 횟집강남역 막걸리강남역 초밥강남역 야장강남역 버스강남역 주소


['강남역임플란트 치아상실의 대처 방안으로', '강남역 피부과 리쥬란힐러 생생한 전후비교 후기', '강남역임플란트 치아를 잃어 대처하기 위해', '강남역 효뜨/무난했던 쌀국수와 먹을 만했던 볶음밥', '강남역 네일 추천, 집중 케어 네일샵 디자인뷰티플러스', '강남역 속눈썹펌 근처에서 이 가격 못찾았어요.', '강남역 한의원 교통사고 발생했을때', '감동 이상인 강남역 고기집', '강남역 맛집 역삼역 점심으로 중식 먹기! 신복면관 우육탕', '취향저격인 강남역 고기집', '강남역피부과추천 악센토 받고 주근깨 박멸한 후기', '강남역 피부관리, 리프팅레이저 후 수분관리 코어킴스킨', '강남역 샤브샤브 맛집 미미옥 신논현역 한식 맛집', '강남역헬스장 HIIT 프로그램으로 운동해보기!', '강남역두피관리 닥터모락에서 말끔히 관리받았어요', '강남역 점심 혼밥 맛집 무한미니훠궈에서 살덜찌게 먹는방법', '강남역 삼겹살 가성비 코스요리처럼 구워주는 그릴링서비스 유니네고깃간', '강남역 한식 추천 청간막국수', '강남역 사무실 임대 케이스퀘어 강남2', '강남역 치과 추천 사례별 치료과정', '강남역 사무실 임대 광일프라자', '인상깊은 강남역 삼겹살', '강남역 피부관리 헤세드스킨케어 여드름관리 추천하는이유!', '강남역 타로상담 해주는 칵테일바 타로맨스', "강남역 삼겹살 가성비 맛집 '유니네 고깃간'", '강남역 치과 추천 임플란트 식립과정', '강남역 튠페이스 합리적 가격의 청담 피부과 처럼', '[도키메이테.강남역.디너.#3] 강남역 최근접 오마카세 스시야', '강남역 소고기 맛집 한우부티크 점심에도 실속있게.', '강남역 소고기 맛집 | 

In [6]:
print(len(blog_titles),len(blog_urls))

1749 1749


- blog 본문 가져오기

In [ ]:
# Chrome 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
blog_url =
driver.get(blog_url)
time.sleep(1)


for i in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(0.05)
    
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")





In [91]:
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
end_page=total_num//7+1
end_page

1689

In [67]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate=2024-04-26&endDate=2024-05-03&keyword=%EA%B0%95%EB%82%A8%EC%97%AD"
driver.get(url)
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
contents_lst = soup.select('div.info_post > div.desc > a.desc_inner')


titles=[]
urls=[]

for i in contents_lst:
    href=i.attrs['href']
    title=i.get_text().replace('\n','').strip()
    titles.append(title)
    urls.append(href)    
    
print(titles)
print(urls)

['강남역 피부과 리쥬란힐러 생생한 전후비교 후기', '강남역임플란트 치아를 잃어 대처하기 위해', '강남역 효뜨/무난했던 쌀국수와 먹을 만했던 볶음밥', '강남역 네일 추천, 집중 케어 네일샵 디자인뷰티플러스', '강남역 속눈썹펌 근처에서 이 가격 못찾았어요.', '강남역 한의원 교통사고 발생했을때', '감동 이상인 강남역 고기집']
['https://blog.naver.com/ycs7117/223435139372', 'https://blog.naver.com/vineyardk/223433935912', 'https://blog.naver.com/sagak815/223425757331', 'https://blog.naver.com/keiwj/223432036619', 'https://blog.naver.com/tgsvckim/223430022615', 'https://blog.naver.com/yoonaqut/223427883720', 'https://blog.naver.com/eaenvy77/223428308046']


In [46]:
def temp(driver):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')

    # 연관검색어 리스트에 담기
    return soup.select('div > div.detail_box > div.title_area')

In [47]:
# 키워드 입력
query = input("검색어 : ")



# Chrome 실행
driver = webdriver.Chrome()
# url = "https://section.blog.naver.com/" 
url = f'https://search.naver.com/search.naver?ssc=tab.blog.all&query={query}&sm=tab_opt&nso=so%3Ar%2Cp%3A1w'
driver.get(url)
time.sleep(2)


sub_keywords_data1 = temp(driver)

while True: 
    try: 
        element = driver.find_element(By.TAG_NAME,'body')
        element.send_keys(Keys.END)
        time.sleep(3)
        sub_keywords_data2 = temp(driver)
        
        if len(sub_keywords_data1) == len(sub_keywords_data2):
            break 
        
        # 초기화 
        sub_keywords_data1 = sub_keywords_data2
        
    except : 
        print('finish')
        break 
    

# 연관검색어 리스트에 담기
sub_keywords_data = temp(driver)
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print(sub_keywords)
        

[' 강남역 효뜨/무난했던 쌀국수와 먹을 만했던 볶음밥 ', ' 강남역 와인바 띠엘로 모임장소 후기 ', ' 취향저격인 강남역 고기집 ', ' 강남역 맛집 역삼역 점심으로 중식 먹기! 신복면관 우육탕 ', ' 강남역 소고기맛집이자 가족외식 장소 추천, 한우부티크 후기 ', ' 장인닭갈비 강남 강남역 저녁 밥집 추천 맛있게 먹을 수 있는 곳! ', ' 강남역 샤브샤브 맛집 미미옥 신논현역 한식 맛집 ', ' [강남] 프라이빗한 룸식당 추천! 강남역 소고기 | 한우부티크 ', ' 강남역스시맛집 - 합리적인 가격으로 맛있고 다양한 초밥을 즐길 수 있는 우리동네스시신조 강남점 ', ' 감동 이상인 강남역 고기집 ', ' 강남역 소고기 룸식당 한우부티크 런치 꽃등심, 점심 회식장소로 좋아 ', ' 강남역두피관리 닥터모락에서 말끔히 관리받았어요 ', ' 강남역 점심 혼밥 맛집 무한미니훠궈에서 살덜찌게 먹는방법 ', ' 강남역 마사지 : 국가공인안마센터 참손길지압힐링센터 강남점 ', ' 강남역 곱창 고소함 가득 소곱창에 가성비까지 육연타 ', ' 강남역 소고기 일편등심 강남점 : 법카 회식장소 강남역 맛집 ', ' 강남역 횟집 룸식당 강남역 회 맛집 잡어와묵은지 ', ' 강남역 한식 추천 청간막국수 ', ' 강남역카페 셀릭티드닉스 초코바나나티라미수+아메리카노 ', ' [도키메이테.강남역.디너.#3] 강남역 최근접 오마카세 스시야 ', ' 강남역 무한리필 추천 라장 양갈비 양꼬치 ', " 강남역 삼겹살 가성비 맛집 '유니네 고깃간' ", ' 강남역 소고기 맛집 한우부티크 평일 디너 ', ' 인상깊은 강남역 삼겹살 ', ' [서울 맛집] #404 강남진해장 - 강남역 24시 곱창전골 국밥 ', ' 강남역 소고기 중요한 자리에 좋은 식당 강남역 5번출구 한우부티크 _ 룸식당 ', ' 강남브런치카페 프라텔리 프렌치토스트와 커피가 맛있는 강남역맛집 추천 ', ' 로리스 더 프라임립 강남역 맛집 추천 디너 코스요리 후기 ', ' 어퍼앤언더 │ 신논현역 디저트 카페 강남역CGV근처 넓

In [ ]:
def infinite_loop():
    #스크롤 내리기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_page_height == last_page_height:
            time.sleep(1.0)
            if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

In [17]:
import requests  
url = 'https://blog.naver.com/yoon0_jo/223426271987'

response = requests.get(url)  
html = response.text
print(html)








<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta http-equiv="Pragma" content="no-cache"/>
<meta http-equiv="Expires" content="-1"/>
<meta name="robots" content="noindex,follow"/>
<meta name="referrer" content="always"/>
<meta http-equiv="content-type" content="text/html;charset=UTF-8"/>
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<link rel="shortcut icon" type="image/x-icon" href="/favicon.ico?3" />
<link rel="alternate" type="application/rss+xml" href="https://rss.blog.naver.com/yoon0_jo.xml" title="RSS feed for yoon0_jo Blog"/>
<link rel="wlwmanifest" type="application/wlwmanifest+xml" href="https://blog.naver.com/NBlogWlwLayout.naver?blogId=yoon0_jo" />




<title>어쨌든! 해보자 가보자 즐기자ᕦ(ㅇ,ㅇ)ᕤ : 네이버 블로그</title>
</head>
<script type="text/javascript" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-527057374_https.js" charset="UTF-8"></scrip

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


driver = webdriver.Chrome()
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
driver = webdriver.Chrome(options = options)
# options = webdriver.ChromeOptions()
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
url = 'https://blog.naver.com/yoon0_jo/223426271987'
driver.get(url)
time.sleep(1)


blog_titles = []
blog_contents = []
blog_hashtag = []



# Pagedown
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.0)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_page_height == last_page_height:
        time.sleep(1.0)
        if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
            break
    else:
        last_page_height = new_page_height



# n = 0
# while n < 100: 
#     try: 
#         element = driver.find_element(By.ID,'body')
#         element.send_keys(Keys.END)
#         n = n+1
#     except : 
#         print('finish')
#         break 
    
    
html = driver.page_source  
soup = BeautifulSoup(response.text, 'html.parser') 
WebDriverWait(driver, 10).until(EC.element_located_to_be_selected((By.CLASS_NAME,"se-main-container")))
content = driver.find_element(By.CLASS_NAME,"se-main-container")
print(content)
# blog_contents.append(content)

# # print(blog_titles)
# # print(blog_contents)
    
# # for blog_url in urls : 
# #     driver.get(blog_url) 
# #     time.sleep(1)
    
# #     # Pagedown
# #     n = 0
# #     while n < 100: 
# #         try: 
# #             element = driver.find_element(By.TAG_NAME,'body')
# #             element.send_keys(Keys.END)
# #             n = n+1
# #         except : 
# #             print('finish')
# #             break 
        
    
# #     html = driver.page_source  
# #     soup = BeautifulSoup(html,'lxml')
# #     title = driver.find_element(By.CSS_SELECTOR, '#SE-e65b5939-03cd-11ef-bec1-cb3307d44077').text
# #     content = driver.find_elements(By.CLASS_NAME, 'se-component-content').text
# #     blog_titles.appen(title)
# #     blog_contents(content)
    

# # print(blog_titles)
# # print(blog_contents)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF701AD1502+60802]
	(No symbol) [0x00007FF701A4AC02]
	(No symbol) [0x00007FF701907CE4]
	(No symbol) [0x00007FF701956D4D]
	(No symbol) [0x00007FF701956E1C]
	(No symbol) [0x00007FF70199CE37]
	(No symbol) [0x00007FF70197ABBF]
	(No symbol) [0x00007FF70199A224]
	(No symbol) [0x00007FF70197A923]
	(No symbol) [0x00007FF701948FEC]
	(No symbol) [0x00007FF701949C21]
	GetHandleVerifier [0x00007FF701DD411D+3217821]
	GetHandleVerifier [0x00007FF701E160B7+3488055]
	GetHandleVerifier [0x00007FF701E0F03F+3459263]
	GetHandleVerifier [0x00007FF701B8B846+823494]
	(No symbol) [0x00007FF701A55F9F]
	(No symbol) [0x00007FF701A50EC4]
	(No symbol) [0x00007FF701A51052]
	(No symbol) [0x00007FF701A418A4]
	BaseThreadInitThunk [0x00007FFD744B257D+29]
	RtlUserThreadStart [0x00007FFD7582AA48+40]
